In [2]:
ifc_path = r"C:\Users\eibre\OneDrive - Norconsult Group\Documents\NB3_ARK.ifc"

import numpy as np
import ifcopenshell

# Edit placements and delete the ones that places spaces
def a2p(o, z, x):
    y = np.cross(z, x)
    r = np.eye(4)
    r[:-1,:-1] = x,y,z
    r[-1,:-1] = o
    return r.T

def get_axis2placement(plc):
    z = np.array(plc.Axis.DirectionRatios if plc.Axis else (0,0,1))
    x = np.array(plc.RefDirection.DirectionRatios if plc.RefDirection else (1,0,0))
    o = plc.Location.Coordinates
    return a2p(o,z,x)

def get_local_placement(plc):
    if plc is None:
        return np.eye(4)
    if plc.PlacementRelTo is None:
        parent = np.eye(4)
    else:
        parent = get_local_placement(plc.PlacementRelTo)

    # OR
    # return np.dot(get_axis2placement(plc.RelativePlacement), parent)
    # OR
    return np.dot(parent, get_axis2placement(plc.RelativePlacement))

def create_axis2placement(f, x, z, o):
    x = f.createIfcDirection(x)
    z = f.createIfcDirection(z)
    o = f.createIfcCartesianPoint(o)
    axes = f.createIfcAxis2Placement3D(o, z, x)

    return axes

import ifcopenshell
print("Opening file at {} .....".format(ifc_path))
f = ifcopenshell.open(ifc_path)

spaces = f.by_type('IfcSpace')
print("Iterating over {} spaces and iterates through the elements contained by the space and associates them with the spaces container".format(len(spaces)))
for space in spaces:
    container = space.Decomposes[0].RelatingObject
    if len(space.ContainsElements) > 0:
        for contained_element in space.ContainsElements[0].RelatedElements:
            contained_element.ContainedInStructure[0].RelatingStructure = container


print("Iterating over each spaces ObjectPlacement each element which is placed relative to it location and changes it to the container of the space.".format(len(local_placements)))     
for space in spaces:
    lp_space = space.ObjectPlacement
    R_space = get_axis2placement(lp_space.RelativePlacement)
    plcRelTo_space = lp_space.PlacementRelTo

    for lp_ref in lp_space.ReferencedByPlacements:
        R_ref = get_axis2placement(lp_ref.RelativePlacement)

        # OR
        # R_new = R_ref.dot(R_space)
        # OR
        R_new = R_space.dot(R_ref)

        relPlc_new = create_axis2placement(
            f,
            x=R_new[:-1, 0].tolist(),
            z=R_new[:-1, 2].tolist(),
            o=R_new[:-1, -1].tolist()
        )
        plc_new = f.createIfcLocalPlacement(plcRelTo_space, relPlc_new)
        lp_ref.PlacesObject[0].ObjectPlacement = plc_new

    #f.remove(lp_space)

print("Deleting {} spaces".format(len(spaces)))
for space in f.by_type("ifcspace"):
    f.remove(space)

save_path = ifc_path[:-4]+"-space-rem.ifc"
print("Saving file to {} ...".format(save_path))
f.write(save_path)

import os
os.startfile(save_path)

print("Done!")

Opening file at C:\Users\eibre\OneDrive - Norconsult Group\Documents\NB3_ARK.ifc .....
Iterating over 844 spaces and iterates through the elements contained by the space and associates them with the spaces container
Iterating over each spaces ObjectPlacement each element which is placed relative to it location and changes it to the container of the space.
Deleting 844 spaces
Saving file to C:\Users\eibre\OneDrive - Norconsult Group\Documents\NB3_ARK-space-rem.ifc ...
Done!
